In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import API_KEY
gmaps.configure(api_key=API_KEY)

In [12]:
#Store Part I results (from WeatherPy) into DataFrame and Load the csv exported in Part I to a DataFrame.

weather_csv_file = "../WeatherPy/Output/weather_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
0,0,victoria,HK,47,1632328632,82,22.2855,114.1577,87.33,11.86
1,1,saint-philippe,RE,45,1632328737,74,-21.3585,55.7679,69.35,11.52
2,2,busselton,AU,2,1632328738,92,-33.6500,115.3333,56.08,10.27
3,3,qaanaaq,GL,12,1632328738,60,77.4840,-69.3632,14.29,6.40
4,4,medak,IN,100,1632328738,93,18.0333,78.2667,77.90,5.14



Out[2]:
City	Cloudiness	Country	Date	Humidity	Latitude	Longitude	Max Temp	Wind Speed
0	hasaki	75.0	JP	1.580018e+09	86.0	35.73	140.83	48.00	5.82
1	oktyabrskoye	0.0	RU	1.580018e+09	85.0	43.06	44.74	19.00	6.71
2	atuona	37.0	PF	1.580018e+09	76.0	-9.80	-139.03	81.00	14.47
3	rikitea	24.0	PF	1.580018e+09	78.0	-23.12	-134.97	78.73	11.10
4	bredasdorp	69.0	ZA	1.580018e+09	77.0	-34.53	20.04	63.00	12.75

In [30]:

# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.

# Configure gmaps with API key.
gmaps.configure(api_key=API_KEY)

# Also, handle NaN values
weather_df = weather_df.dropna()

# Convert Humidity to float and store
humidity = weather_df["Humidities"].astype(float)
humidity

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lats", "Lngs"]].astype(float)
locations
# Create a humidity Heatmap layer
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
fig
# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidities"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig
#plt.savefig("figures/figure15.png")


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))

In [14]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max_temps"] < 80) & (filtered_weather_df["Max_temps"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind_speeds"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
202,202,resistencia,AR,0,1632328606,58,-27.4606,-58.9839,70.09,8.05
238,238,yichang,CN,0,1632328814,78,30.7144,111.2847,75.76,2.51
311,311,saint-denis,FR,0,1632328837,54,48.9167,2.3333,70.23,1.99
454,454,vallenar,CL,0,1632328887,22,-28.5708,-70.7581,74.44,6.20
481,481,damghan,IR,0,1632328895,19,36.1683,54.3480,78.85,7.49
482,482,changde,CN,0,1632328896,64,29.0464,111.6783,74.75,0.81


In [15]:
hotel_df =filtered_weather_df

hotel_df['Hotel Name'] = np.nan
hotel_df
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lats']}, {row['Lngs']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": API_KEY
        
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df  
narrowed_city_df.dropna()


,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds,Hotel Name
202,202,resistencia,AR,0,1632328606,58,-27.4606,-58.9839,70.09,8.05,Amérian Hotel Casino Gala
238,238,yichang,CN,0,1632328814,78,30.7144,111.2847,75.76,2.51,Crowne Plaza Yichang
311,311,saint-denis,FR,0,1632328837,54,48.9167,2.3333,70.23,1.99,L'imprimerie hôtel
454,454,vallenar,CL,0,1632328887,22,-28.5708,-70.7581,74.44,6.20,Hotel Cecil
481,481,damghan,IR,0,1632328895,19,36.1683,54.3480,78.85,7.49,Danesh Hotel
482,482,changde,CN,0,1632328896,64,29.0464,111.6783,74.75,0.81,Sheraton Changde Wuling Hotel


In [16]:

# Assign the marker layer to a variable
locations = narrowed_city_df[["Lats", "Lngs"]]
markers = gmaps.marker_layer(locations)


In [17]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))